##Install home-made packages and pytest

### Install home-made packages

In [1]:
!python -m pip install -e .

Obtaining file:///content
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for chapter-6 (pyproject.toml) ... done
  Created wheel for chapter-6: filename=chapter_6-0.1-py2.py3-none-any.whl size=987 sha256=4ccc1dada86af171ee4783a5c9821dd9ea1b640b5c16e73059b9741f9c7b60ee
  Stored in directory: /tmp/pip-ephem-wheel-cache-dyq2f25o/wheels/e8/d3/96/0e8c7135806cbda4db28d12fc8d710e5e4f66ced1411163e67
Successfully built chapter-6


### Install pytest

In [2]:
!python -m pip install pytest

### Run tests

In [3]:
!python -m pytest tests

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-8.3.4, pluggy-1.5.0
rootdir: /content
configfile: setup.cfg
plugins: typeguard-4.4.1, anyio-3.7.1
collected 17 items                                                                                 

tests/test_exercise_6_1.py .......                                                           [ 41%]
tests/test_exercise_6_2.py .........                                                         [ 94%]
tests/test_exercise_6_3.py .                                                                 [100%]

======================================== 17 passed in 0.32s ========================================


## Code

In [ ]:
class ConvergenceError(Exception):
    """Exception raised if a solver fails to converge."""

    pass

## Newton-Raphson

In [ ]:
import numpy as np

In [ ]:
def newton_raphson(f, df, x_0, eps=1.0e-5, max_its=20):
    """Solve a nonlinear equation using Newton-Raphson iteration.

    Solve f==0 using Newton-Raphson iteration.

    Parameters
    ----------
    f : function(x: float) -> float
        The function whose root is being found.
    df : function(x: float) -> float
        The derivative of f.
    x_0 : float
        The initial value of x in the iteration.
    eps : float
        The solver tolerance. Convergence is achieved when abs(f(x)) < eps.
    max_its : int
        The maximum number of iterations to be taken before the solver is taken
        to have failed.

    Returns
    -------
    float
        The approximate root computed using Newton iteration.
    """

    def g(x):
        """ return the iteration function g(x) for given f(x) and f'(x) """
        return x - f(x) / df(x)

    r = -1
    x_r = x_0
    x_r_plus_1 = g(x_r)

    while abs(x_r_plus_1 - x_r) > eps and r < max_its:
        r = r + 1
        x_r = x_r_plus_1
        x_r_plus_1 = g(x_r)
    if r >= max_its:
        raise  ConvergenceError(f'max_its of {max_its} has been exceeded')
    return x_r_plus_1

###Correct roor expected

In [ ]:
root = newton_raphson(lambda x: x**2 - 5*x + 2, lambda x: 2*x - 5, 0)
print(f'Root_1: {root:.5f}')

Root_1: 0.43845


In [ ]:
newton_raphson(lambda x: np.cos(x) - x, lambda x: -np.sin(x) - 1, 1, 1e-10, 4) #, 0.7390851332)

0.7390851332151607

In [ ]:
newton_raphson(lambda x: np.sin(np.exp(x)) - 1, lambda x: np.exp(x) *
     np.cos(np.exp(x)), 0.5, 1e-10, 25) #, 0.45158270719343396)

0.45158270719343396

In [ ]:
newton_raphson(lambda x: x**2 - 1, lambda x: 2 * x, 2, 1e-5, 5) #, 1.00000004646114741)

1.000000000000001

### Exception expected

In [ ]:
newton_raphson(lambda x: np.cos(x) - x, lambda x: -np.sin(x) - 1, 1, 1e-10, 2)

ConvergenceError: max_its of 2 has been exceeded

In [ ]:
newton_raphson(lambda x: np.sin(np.exp(x)) -
     1, lambda x: np.exp(x) *
     np.cos(np.exp(x)), 0.5, 1e-20, 20)

ConvergenceError: max_its of 20 has been exceeded

In [ ]:
newton_raphson(lambda x: x**(1 / 3), lambda x: (1 / 3) * x**(-2 / 3), 1, 1e-5, 5)

ConvergenceError: max_its of 5 has been exceeded

In [ ]:
newton_raphson(lambda x: x**2 - 1, lambda x: 2 * x, 2, 1e-5, 1)

ConvergenceError: max_its of 1 has been exceeded

##Bisection method

In [ ]:
def bisection(f, a, b, eps=1.0e-5, max_its=20):
    """Solve a nonlinear equation using bisection.

    Solve f==0 using bisection starting with the interval [x_0, x_1]. f(x_0)
    and f(x_1) must differ in sign.

    Parameters
    ----------
    f : function(x: float) -> float
        The function whose root is being found.
    a : float
        The left end of the initial bisection interval.
    b : float
        The right end of the initial bisection interval.
    eps : float
        The solver tolerance. Convergence is achieved when abs(f(x)) < eps.
    max_its : int
        The maximum number of iterations to be taken before the solver is taken
        to have failed.

    Returns
    -------
    float
        The approximate root computed using bisection.
    """
    its = 0
    c = (a + b) / 2
    while abs(f(c)) > eps:
        # ValueError checked during while loop
        if f(a) * f(b) >= 0:
            raise ValueError(f'f(x_0) and f(x_1) are not of the same sign')
        c = (a + b) / 2
        if f(c) == 0:
            break
        elif f(a) * f(c) < 0:
            b = c
        else:
            a = c
        c = (a + b) / 2
        its += 1
    # ConvergenceError checked after while loop terminates
    if its >= max_its:
        raise  ConvergenceError(f'max_its of {max_its} has been exceeded')
    return c

###Root expected

In [ ]:
bisection(lambda x: np.cos(x) - x, 0, 1, 1e-5, 17) #, 0.7390899658203125)

0.7390899658203125

In [ ]:
bisection(lambda x: 4 * x**3 - 1, -1, 2, 1e-5, 20) #, 0.6299591064453125)

0.6299591064453125

In [ ]:
bisection(lambda x: x**2 - 1, 0.825, 8.125, 1e-5, 25) #, 0.9999993324279786)

0.9999993324279786

###ConvergenceError exception expected

In [ ]:
bisection(lambda x: np.cos(x) - x, 0, 1, 1e-5, 2) #, 0.7390899658203125)

ConvergenceError: max_its of 2 has been exceeded

In [ ]:
bisection(lambda x: 4 * x**3 - 1, -1, 2, 1e-5, 4) #, 0.6299591064453125)

ConvergenceError: max_its of 4 has been exceeded

In [ ]:
bisection(lambda x: x**2 - 1, 0.825, 8.125, 1e-5, 6) #, 0.9999993324279786)

ConvergenceError: max_its of 6 has been exceeded

###ValueError exception expected

In [ ]:
bisection(lambda x: np.cos(x) - x, 0.7, 0.73, 1e-5, 20) #, 0.7390899658203125)

ValueError: f(x_0) = 0.064842 and f(x_1) = 0.015174

In [ ]:
bisection(lambda x: 4 * x**3 - 1, 1, 2, 1e-5, 20) #, 0.6299591064453125)

ValueError: f(x_0) and f(x_1) are not of the same sign

In [ ]:
bisection(lambda x: x**2 - 1, 1.825, 8.125, 1e-5, 25) #, 0.9999993324279786)

ValueError: f(x_0) = 2.3306 and f(x_1) = 65.016

##Solve

In [ ]:
def solve(f, df, x_0, x_1, eps=1.0e-5, max_its_n=20, max_its_b=20):
    """Solve a nonlinear equation.

    solve f(x) == 0 using Newton-Raphson iteration, falling back to bisection
    if the former fails.

    Parameters
    ----------
    f : function(x: float) -> float
        The function whose root is being found.
    df : function(x: float) -> float
        The derivative of f.
    x_0 : float
        The initial value of x in the Newton-Raphson iteration, and left end of
        the initial bisection interval.
    x_1 : float
        The right end of the initial bisection interval.
    eps : float
        The solver tolerance. Convergence is achieved when abs(f(x)) < eps.
    max_its_n : int
        The maximum number of iterations to be taken before the newton-raphson
        solver is taken to have failed.
    max_its_b : int
        The maximum number of iterations to be taken before the bisection
        solver is taken to have failed.

    Returns
    -------
    float
        The approximate root.
    """
    try:
        root_n = newton_raphson(f, df, x_0, eps, max_its_n)
    except ConvergenceError:
        try:
            root_b = bisection(f, x_0, x_1, eps, max_its_b)
        except ValueError:
            pass
        else:
            print('Bisection')
            return root_b
    else:
        print(f'Newton')
        return root_n

###Caught by Newton

In [ ]:
solve(lambda x: np.cos(x) - x, lambda x: -np.sin(x) - 1, 0, 1, 1e-5, 15, 20) #, 0.7390899658203125)

Newton


0.7390851332151607

###Caught by Bisection

In [ ]:
solve(lambda x: np.cos(x) - x, lambda x: -np.sin(x) - 10, 0, 1, 1e-5, 15, 20) #, 0.7390899658203125)

Bisection


0.7390899658203125